In [1]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from constants import (
    TARGET_SPARSITY_LOW, TARGET_SPARSITY_MID, TARGET_SPARSITY_HIGH,
    BATCH_SIZE_CNN, BATCH_SIZE_VIT, BATCH_SIZE_LLM,
    EPOCHS_SMALL_MODEL, EPOCHS_LARGE_MODEL, EPOCHS_VIT
)
from utils import get_device, get_num_workers, load_weights, print_statistics
from unstructured_pruning import check_model_sparsity, check_sparsity_distribution
from trainer import TrainingArguments, Trainer
from bacp import BaCPTrainingArguments, BaCPTrainer

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


# DistilBERT

In [3]:
MODEL_NAME = "distilbert-base-uncased"
MODEL_TASK = "wikitext2"
TRAIN = True

## Baseline Accuracies

In [9]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-5),
    scheduler_type='linear_with_warmup',
    epochs=50,
    learning_type="baseline",
    db=False,
)
trainer = Trainer(training_args=training_args)
if False:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (adamw, 5e-05)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 355 and total steps: 3550
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     64.00%
  Perplexity:   5.743

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.0000 (0.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        baseline
  Batch Size:           64
  Learning Rate:        5e-05
  Optimizer:            adamw
  Epochs:               50

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## Pruning Accuracies

### Magnitude Prune

In [21]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if False:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.95_pruning.pt
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     47.98%
  Perplexity:   27.106

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [22]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/magnitude_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Training epoch [1/5]: Avg Loss: 2.1292 | Avg Accuracy: 58.90 | Model Sparsity: 0.4734
Avg Perplexity: 8.251

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.9564 | Avg Accuracy: 59.13 | Model Sparsity: 0.4734
Avg Perplexity: 8.067

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 2.2232 | Avg Accuracy: 45.52 | Model Sparsity: 0.97
Avg Perplexity: 31.533

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 2.1948 | Avg Accuracy: 45.56 | Model Sparsity: 0.97
Avg Perplexity: 31.921

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 2.1798 | Avg Accuracy: 45.93 | Model Sparsity: 0.97
Avg Perplexity: 31.103

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 2.1564 | Avg Accuracy: 45.40 | Model Sparsity: 0.97
Avg Perplexity: 31.313

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     45.43%
  Perplexity:   31.024

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [23]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/magnitude_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Training epoch [1/5]: Avg Loss: 2.1525 | Avg Accuracy: 58.95 | Model Sparsity: 0.4831
Avg Perplexity: 8.258

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 1.9669 | Avg Accuracy: 58.62 | Model Sparsity: 0.4831
Avg Perplexity: 8.309



...

Recovery epoch [7/10]: Avg Loss: 2.6338 | Avg Accuracy: 41.20 | Model Sparsity: 0.99
Avg Perplexity: 44.666

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 2.5991 | Avg Accuracy: 41.06 | Model Sparsity: 0.99
Avg Perplexity: 45.093



Recovery epoch [9/10]: Avg Loss: 2.5738 | Avg Accuracy: 40.99 | Model Sparsity: 0.99
Avg Perplexity: 45.583



Recovery epoch [10/10]: Avg Loss: 2.5415 | Avg Accuracy: 41.39 | Model Sparsity: 0.99
Avg Perplexity: 44.734

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_magnitude_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     41.43%
  Perplexity:   45.728

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### SNIP-it Prune

In [24]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Training epoch [1/5]: Avg Loss: 3.1116 | Avg Accuracy: 54.65 | Model Sparsity: 0.4636
Avg Perplexity: 10.971

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 2.2774 | Avg Accuracy: 55.46 | Model Sparsity: 0.4636
Avg Perplexity: 10.289

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 2.0946 | Avg Accuracy: 45.23 | Model Sparsity: 0.95
Avg Perplexity: 34.098



Recovery epoch [8/10]: Avg Loss: 2.0720 | Avg Accuracy: 45.48 | Model Sparsity: 0.95
Avg Perplexity: 32.527

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 2.0649 | Avg Accuracy: 45.18 | Model Sparsity: 0.95
Avg Perplexity: 34.410



Recovery epoch [10/10]: Avg Loss: 2.0418 | Avg Accuracy: 45.55 | Model Sparsity: 0.95
Avg Perplexity: 33.867

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     45.71%
  Perplexity:   33.934

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [25]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Training epoch [1/5]: Avg Loss: 3.2142 | Avg Accuracy: 54.11 | Model Sparsity: 0.4734
Avg Perplexity: 11.187

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 2.2967 | Avg Accuracy: 54.88 | Model Sparsity: 0.4734
Avg Perplexity: 10.666

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 2.3090 | Avg Accuracy: 42.98 | Model Sparsity: 0.97
Avg Perplexity: 40.313



Recovery epoch [8/10]: Avg Loss: 2.2772 | Avg Accuracy: 42.78 | Model Sparsity: 0.97
Avg Perplexity: 41.284



Recovery epoch [9/10]: Avg Loss: 2.2473 | Avg Accuracy: 43.18 | Model Sparsity: 0.97
Avg Perplexity: 40.265



Recovery epoch [10/10]: Avg Loss: 2.2419 | Avg Accuracy: 43.02 | Model Sparsity: 0.97
Avg Perplexity: 40.895

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     42.28%
  Perplexity:   40.321

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [8]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/distilbert-base-uncased/wikitext2/pruning/snip_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Training epoch [1/5]: Avg Loss: 3.2108 | Avg Accuracy: 54.77 | Model Sparsity: 0.4831
Avg Perplexity: 10.788

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 2.2834 | Avg Accuracy: 56.11 | Model Sparsity: 0.4831
Avg Perplexity: 9.986

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 2.8021 | Avg Accuracy: 38.41 | Model Sparsity: 0.99
Avg Perplexity: 61.215

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 2.7686 | Avg Accuracy: 37.81 | Model Sparsity: 0.99
Avg Perplexity: 62.193



Recovery epoch [9/10]: Avg Loss: 2.7368 | Avg Accuracy: 38.30 | Model Sparsity: 0.99
Avg Perplexity: 61.324



Recovery epoch [10/10]: Avg Loss: 2.7098 | Avg Accuracy: 38.54 | Model Sparsity: 0.99
Avg Perplexity: 60.417

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_snip_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     37.80%
  Perplexity:   65.203

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### WandA Prune

In [4]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
.weight
model.weight
model.activation.weight
model.distilbert.weight
model.distilbert.embeddings.weight
model.distilbert.embeddings.word_embeddings.weight
model.distilbert.embeddings.position_embeddings.weight
model.distilbert.embeddings.LayerNorm.weight
model.distilbert.embeddings.dropout.weight
model.distilbert.transformer.weight
model.distilbert.transformer.layer.weight
model.distilbert.transformer.layer.0.weight
model.distilbert.transformer.layer.0.attention.weight
model.distilbert.transformer.layer.0.attention.dropout.weight
model.distilbert.transformer.layer.0.attention.q_lin.weight

Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Training Epoch [1/5]:   4%|▍         | 3/71 [00:01<00:25,  2.69it/s, Loss=2.8433, Sparsity=0.4636]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 2.1098 | Avg Accuracy: 45.61 | Model Sparsity: 0.95
Avg Perplexity: 32.791

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 2.0766 | Avg Accuracy: 45.36 | Model Sparsity: 0.95
Avg Perplexity: 32.990



Recovery epoch [9/10]: Avg Loss: 2.0743 | Avg Accuracy: 45.79 | Model Sparsity: 0.95
Avg Perplexity: 33.681

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 2.0494 | Avg Accuracy: 45.56 | Model Sparsity: 0.95
Avg Perplexity: 33.996

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_wanda_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     45.65%
  Perplexity:   33.869

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [6]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
.weight
model.weight
model.activation.weight
model.distilbert.weight
model.distilbert.embeddings.weight
model.distilbert.embeddings.word_embeddings.weight
model.distilbert.embeddings.position_embeddings.weight
model.distilbert.embeddings.LayerNorm.weight
model.distilbert.embeddings.dropout.weight
model.distilbert.transformer.weight
model.distilbert.transformer.layer.weight
model.distilbert.transformer.layer.0.weight
model.distilbert.transformer.layer.0.attention.weight
model.distilbert.transformer.layer.0.attention.dropout.weight
model.distilbert.transformer.layer.0.attention.q_lin.weight

Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Training Epoch [1/5]:   4%|▍         | 3/71 [00:01<00:19,  3.58it/s, Loss=2.4791, Sparsity=0.4734]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 2.2530 | Avg Accuracy: 43.50 | Model Sparsity: 0.97
Avg Perplexity: 38.572



Recovery epoch [8/10]: Avg Loss: 2.2343 | Avg Accuracy: 44.08 | Model Sparsity: 0.97
Avg Perplexity: 38.199



Recovery epoch [9/10]: Avg Loss: 2.2182 | Avg Accuracy: 44.00 | Model Sparsity: 0.97
Avg Perplexity: 39.007



Recovery epoch [10/10]: Avg Loss: 2.1923 | Avg Accuracy: 44.07 | Model Sparsity: 0.97
Avg Perplexity: 38.216

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_wanda_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     44.65%
  Perplexity:   36.423

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [7]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,
    optimizer_type_and_lr=('adamw', 5e-4),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: None
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (adamw, 0.0005)
[TRAINER] Data Initialized for model task: wikitext2
[TRAINER] Batch size: 64
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
.weight
model.weight
model.activation.weight
model.distilbert.weight
model.distilbert.embeddings.weight
model.distilbert.embeddings.word_embeddings.weight
model.distilbert.embeddings.position_embeddings.weight
model.distilbert.embeddings.LayerNorm.weight
model.distilbert.embeddings.dropout.weight
model.distilbert.transformer.weight
model.distilbert.transformer.layer.weight
model.distilbert.transformer.layer.0.weight
model.distilbert.transformer.layer.0.attention.weight
model.distilbert.transformer.layer.0.attention.dropout.weight
model.distilbert.transformer.layer.0.attention.q_lin.weight

Training Epoch [1/5]:   0%|          | 0/71 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Training Epoch [1/5]:   4%|▍         | 3/71 [00:01<00:19,  3.57it/s, Loss=2.8648, Sparsity=0.4831]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 2.5552 | Avg Accuracy: 40.66 | Model Sparsity: 0.99
Avg Perplexity: 49.659



Recovery epoch [8/10]: Avg Loss: 2.5408 | Avg Accuracy: 41.14 | Model Sparsity: 0.99
Avg Perplexity: 47.604

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 2.5139 | Avg Accuracy: 40.63 | Model Sparsity: 0.99
Avg Perplexity: 48.296



Recovery epoch [10/10]: Avg Loss: 2.4914 | Avg Accuracy: 40.63 | Model Sparsity: 0.99
Avg Perplexity: 49.430

[TRAINER] Loading weights: ./research/distilbert-base-uncased/wikitext2/distilbert-base-uncased_wikitext2_wanda_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     41.03%
  Perplexity:   48.119

Model Information:
------------------------------
  Total Parameters:     66,985,530
  Trainable Parameters: 66,985,530
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                distilbert-base-uncased
  Task:                 wikitext2
  Learning Type:        pruning
  Batch Size:           64
  Learning Rate:        0.0005
  Optimizer:            adamw
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## BaCP Accuracies

### Magnitude Pruning

In [ ]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### SNIP-it Prune

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### WandA Prune

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [0]:
trained_model_path = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_LLM,   
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=trained_model_path,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args)
if TRAIN:
    bacp_trainer.train()
    
# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 1e-3),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.save_path,
    epochs=50,
    pruner=bacp_trainer.get_pruner(),
    finetune=True,
    learning_type="bacp_finetune",
    db=False
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)